# Bias of Moving Averages
If perhaps it's not obvious: simple moving averages trail the data by approximately half of their window sizes.

Imagine a linearly increasing trend. The average of the last $k$ samples (i.e. $p_{n - k + 1}, p_{n - k + 2}, \ldots, p_{n}$), will be lower than the current value. In order to calculate the 'true' average, that equals the current value, you have to centre it about that point (i.e. $k_{odd} = 2m + 1$, averaging $p_{n - m}, \ldots, p_{n + m}$, or $k_{even} = 2m$, averaging $p_{n - m + 1}, \ldots, p_{n + m}$).

This way, the centre of the calculation has moved from $n - (k - 1)/2$, to $n$ or $n + 1/2$, removing the lag by using information from the future. What you have now is a set of slightly biased* labels.

That bias is clearly visible when plotting a square wave. The centered moving average overshoots the level just before the jump and undershoots the new level just after, because the averaging window spans both sides of the jump. Ideally, the labels would represent the underlying trend (match the square wave) perfectly.

In addition, if the data is cyclical, the window size needs to be calibrated to its frequency to correctly capture trends. If you're attempting to extract the linear trend $at + b$ from the linearly trending sine wave $y(t) = at + b + \sin(\omega t)$, the window should be long enough to average out the cyclical component, but short enough to avoid distorting the underlying trend. In practice, setting the window to roughly an integer multiple of the cycle period ensures that the oscillations cancel out while still capturing the slow trend: $c * 2\pi / \omega \approx \text{window}$.

\* A centered moving average is a convolution with a boxcar kernel. It's exact for polynomials of degree $\leq$ 1, and biased for degree $\geq$ 2. Similarly, Savitzky–Golay filters are convolutions derived from local polynomial least-squares fits. Increasing the polynomial order reduces bias on higher-degree trends but increases variance, as the filter becomes more sensitive to noise in the data.

**Author:** puzzle

**Created:** 2026-02-25

**Modified:** 2026-02-26


In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
# from scipy.signal import savgol_filter

In [2]:
from jfmi.plot.utilities import load_plotly_templates

In [3]:
from jfmi.plot.templates import COLOURS

In [4]:
load_plotly_templates()

In [5]:
sr_t = pd.Series(np.arange(0, 300))

sr_linear_trend = pd.Series(2 * sr_t)
sr_accelerating_trend = pd.Series(2 * (sr_t**1.5))

In [6]:
segment_length = 100

sr_square_wave = pd.Series(
    segment_length * [0] + segment_length * [1] + segment_length * [0]
)

In [7]:
amplitude = 20
frequency = 15 / len(sr_t)  # 15 cycles per sample

sr_sine_wave = pd.Series(amplitude * np.sin(2 * np.pi * frequency * sr_t))
sr_sine_wave_true = pd.Series(np.zeros(len(sr_t)))

In [8]:
sr_sine_wave_trend = sr_linear_trend + sr_sine_wave
sr_sine_wave_trend_true = sr_linear_trend

In [9]:
# Segment 1 - Sine wave
phase_1 = 0

segment_1 = amplitude * np.sin(
    2 * np.pi * frequency * np.arange(segment_length) + phase_1
)

# Segment 2 - Linearly trending sine wave
phase_2 = 2 * np.pi * frequency * (segment_length - 1) + phase_1
trend_2 = 2 * np.arange(segment_length)

segment_2 = trend_2 + amplitude * np.sin(
    2 * np.pi * frequency * np.arange(segment_length) + phase_2
)

# Segment 3 - Sine wave
phase_3 = 2 * np.pi * frequency * (segment_length - 1) + phase_2

segment_3_base = amplitude * np.sin(
    2 * np.pi * frequency * np.arange(segment_length) + phase_3
)

# Align the y-intercept of the third segment with the end of the second segment.
offset_3 = segment_2[-1] - segment_3_base[0]
segment_3 = segment_3_base + offset_3

# Concatenate each segment whilst skipping the first point of segments 2 and 3.
sr_piecewise_sine_wave = pd.Series(
    np.concatenate(
        [
            segment_1,
            segment_2[1:],
            segment_3[1:],
        ]
    )
)
sr_piecewise_sine_wave_true = pd.Series(
    np.concatenate(
        [
            np.zeros(segment_length),
            trend_2,
            np.full(segment_length, offset_3),
        ]
    )
)

In [10]:
# window = 10 + 1  # odd
window = len(sr_t) // 15  # any integer multiple will do

In [11]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=sr_t,
        y=sr_piecewise_sine_wave,
        mode="lines",
        line=dict(color=COLOURS["purple"][4]),
        name="Trend",
    )
)
fig.add_trace(
    go.Scatter(
        x=sr_t,
        y=sr_piecewise_sine_wave_true,
        mode="lines",
        line=dict(color=COLOURS["purple"][7]),
        name="True Trend",
    )
)
fig.add_trace(
    go.Scatter(
        x=sr_t,
        y=sr_piecewise_sine_wave.rolling(window=window).mean(),
        mode="lines",
        line=dict(color=COLOURS["blue"][5]),
        name="Trailing Moving Average",
    )
)
fig.add_trace(
    go.Scatter(
        x=sr_t,
        # Where order 0 is a local constant fit (equivalent to a centered moving
        # average), order 1 is a linear fit, and order 2 is a quadratic fit.
        # y=savgol_filter(sr_piecewise_sine_wave, window_length=window, polyorder=0),
        y=sr_piecewise_sine_wave.rolling(window=window, center=True).mean(),
        mode="lines",
        line=dict(color=COLOURS["orange"][5]),
        name="Centered Moving Average",
    )
)

fig.show()